In [21]:
import yfinance as yf 

import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime
import math
from time import sleep
import quandl
import seaborn as sns
import sys
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
import sys
from scipy.stats import norm
import ezodf
import scipy.optimize as sco
import scipy
import re
import os.path

## Functions


In [22]:
# FUNCTIONS FOR VIEWING SINGLE COMPANY RANKINGS 

def company(company):
    for i in data['Company']:
        if company in i or company in i.lower():
            return data.loc[data['Company'] == i]
    print('No company found')
    
def Company(company):
    for i in data['Company']:
        if company in i or company in i.lower():
            stock = data.loc[data['Company'] == i]
            stock = stock[compact]
            return stock
    print('No company found')

In [23]:
def mod_Z_score(data):
    mean = np.mean(data.iloc[:-1])
    deviation_from_med = np.array(data.iloc[:-1]) - mean
    MAD = np.mean(np.abs(deviation_from_med))
    mod_Z = (data[-1:] - mean)/(MAD * 1.4826)
    return mod_Z

## Data Import and Wrangling


In [24]:
# IMPORT LATEST DATA 

folder = "../equity_data/"
file_name = "Borsdata_2021-11-29.xlsx"

data_tmp = pd.read_excel(os.path.join(folder, file_name), sheet_name="Export")
data_tmp = data_tmp[1:]

data_tmp = data_tmp.rename({'Performance - Perform. 3m' : 'Return 3m','Performance - Perform. 6m' : 'Return 6m',
                            'Performance - Perform. 1y' : 'Return 1y',
                            'Div. Yield - Current': 'Yield', 
                            'Total Equity  - Millions':'Total Equity', 'FCF - Millions': 'FCF','ROE - Current':'ROE',
                            'Volatility - St.Dev. 100d':'Volatility','Market Cap - Current': 'Market Cap', 
                            'ROC - Current':'ROC', 'Tot. Assets - Millions':'Tot. Assets', 
                            'Gross profit - Millions':'Gross profit', 'Assets Turn - Current': 'Assets Turn',
                            'P/FCF - Current':'P/FCF', 'P/E - Current':'P/E', 'P/S - Current':'P/S',
                            'P/B - Current':'P/B','EV/EBIT - Current':'EV/EBIT',
                            'Info - Country' : 'Country','F-Score - Point':'F-Score',
                            'Info - List' : 'List', 'Info - Sector' : 'Sector', 'Info - Industry' : 'Industry',
                            'Info - Ticker' : 'Tick', 'Info - Yahoo':'Yahoo', 'Info - Last Report': 'Last Report',
                           'Volume - Average 50d Mill' : 'Volume'}, axis=1)

# FORMAT FOR VIEWING PLEASURE
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.options.display.float_format = "{:,.2f}".format

In [25]:
# MAKE NUMBERS FLOATS 

data_tmp['Market Cap'] = data_tmp['Market Cap'].replace(',', '') # remove , as 1000 separator

param_list = ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B', 'Assets Turn', 'Gross profit', 'Tot. Assets', 'ROC', 'Yield', 'Market Cap', 'Volatility',
        'ROC', 'ROE', 'FCF', 'Total Equity', 
        'Return 3m', 'Return 6m', 'Return 1y']

for i in param_list: 
    data_tmp[i] = pd.to_numeric(data_tmp[i]) # make every number numeric


data_tmp['FCFROE'] = data_tmp['FCF']/data_tmp['Total Equity'] # calc FCFROE
data_tmp['GPA'] = data_tmp['Gross profit']/data_tmp['Tot. Assets']


data_tmp.loc[(data_tmp['Total Equity'] < 0), ['ROE', 'FCFROE', 'P/B']] = np.nan

In [26]:
### FILTER DATA FOR VOLUME, LIST AND MARKET CAP

data_tmp = data_tmp.loc[data_tmp['Market Cap'] > 400] #Market cap filter


data_tmp = data_tmp.loc[ (data_tmp['List'] != 'Spotlight') 
                        & (data_tmp['List'] != 'NGM')]

#Volume Filter
data_tmp = data_tmp.loc[data_tmp['Volume'] > 0.5] 

# Set to dattime
data_tmp['Last Report'] = pd.to_datetime(data_tmp['Last Report'])

#set new index
data_tmp.index = range(len(data_tmp.index))

In [27]:
print("Companies in data: ",len(data_tmp))
print("Missing Report Dates: ",sum(data_tmp['Last Report'].isna()))



Companies in data:  451
Missing Report Dates:  3


## Calculate Price Based Data from Yahoo Finance

In [28]:
## Get PEAD
data_tmp["EA ret"] = np.nan
data_tmp["EA ret std"] = np.nan
data_tmp['1 Year Volatility'] = np.nan
data_tmp['1 Year MAD'] = np.nan
index = yf.download('^OMXSPI',start='2020-01-01', threads = False, progress = False)
index = index['Close']

error_count = 0
error_list = []

In [29]:
for i in range(len(data_tmp)):
    try:
        stock = yf.download(data_tmp.iloc[i]['Yahoo'],start='2020-01-01', progress = False, threads = False)
        stock = stock['Close']
        import_data = pd.concat([stock, index], axis = 1)
        import_data.columns = ['stock', 'index']
        import_data = import_data.dropna()
        
        # get index of date of report
        idx = import_data.index.get_loc(data_tmp.iloc[i]['Last Report'], method='nearest')
        EA_data = import_data.iloc[idx - 2 : idx +2 ]
        
        stock_idx = stock.index.get_loc(data_tmp.iloc[i]['Last Report'], method='nearest')
        
        EA_ret = (EA_data.pct_change().dropna()+1).cumprod().tail(1)
        pead_ret = float(EA_ret['stock'] - EA_ret['index'])
        data_tmp.loc[i,"EA ret"]= pead_ret
        data_tmp.loc[i, 'EA Volatility'] = np.log(stock.iloc[:idx]/stock.iloc[:idx].shift()).tail(60).std()*252**.5 
        #Get MAD of stock
        data_tmp.loc[i,"1 Year MAD"]= np.log(stock/stock.shift()).tail(252).mad()*252**.5*1.25
        # Get volatility of stock
        data_tmp.loc[i,"1 Year Volatility"]= np.log(stock/stock.shift()).tail(252).std()*252**.5 
       
    except:
        #data_tmp.loc[i,"EA ret"]= np.nan
        #print("ERROR! INDEX: ", i, ', TICKER: ', data_tmp.iloc[i]['Yahoo'])
        error_count = error_count + 1
        error_list.append(i)
    
    


1 Failed download:
- SDIP.ST: No data found, symbol may be delisted


In [30]:
### NA ANALYTICS
print("Amount of Errors: ",error_count )
print("Missing EA rets: ",sum(data_tmp["EA ret"].isna()) )
print("Missing EA volatility: ",sum(data_tmp["EA Volatility"].isna()) )
print("Missing 1 Year volatility: ",sum(data_tmp["1 Year Volatility"].isna()) )

Amount of Errors:  4
Missing EA rets:  4
Missing EA volatility:  4
Missing 1 Year volatility:  4


In [31]:
## Show errors
data_tmp[data_tmp['EA ret'].isna()][['Company', 'Yahoo', 'Last Report', 'EA ret']]

,Company,Yahoo,Last Report,EA ret
20,Aligro Planet SPAC,APAC-SPAC-A.ST,NaT,nan
324,Pfizer,PFE.ST,NaT,nan
364,Sdiptech,SDIP.ST,2021-10-26,nan
382,Smart Wires,GOGRID-SDB.ST,NaT,nan


### TESTING TICKERS ONCE AGAIN, RE ENTER FAILED TICKERS BY HAND

In [32]:
##### TESTING TICKERS ONCE AGAIN, RE ENTER FAILED TICKERS BY HAND #####
j = None
if j != None:
    stock = yf.download(data_tmp.iloc[j]['Yahoo'],start='2020-01-01', progress = False, threads = False)
    stock = stock['Close']
    import_data = pd.concat([stock, index], axis = 1)
    import_data.columns = ['stock', 'index']
    import_data = import_data.dropna()

    # get index of date of report
    idx = import_data.index.get_loc(data_tmp.iloc[j]['Last Report'], method='nearest')
    EA_data = import_data.iloc[idx - 2 : idx +2 ]

    stock_idx = stock.index.get_loc(data_tmp.iloc[j]['Last Report'])

    EA_ret = (EA_data.pct_change().dropna()+1).cumprod().tail(1)
    pead_ret = float(EA_ret['stock'] - EA_ret['index'])
    data_tmp.loc[j,"EA ret"]= pead_ret
    data_tmp.loc[j, 'EA Volatility'] = np.log(stock.iloc[:idx]/stock.iloc[:idx].shift()).tail(60).std()*252**.5 
    #Get MAD of stock
    data_tmp.loc[j,"1 Year MAD"]= np.log(stock/stock.shift()).tail(252).mad()*252**.5*1.25
    # Get volatility of stock
    data_tmp.loc[j,"1 Year Volatility"]= np.log(stock/stock.shift()).tail(252).std()*252**.5

In [33]:
# MANAGE NaNs and NEGATIVE EARNINGS  
# set nan to min
for i in ['Return 3m', 'Return 6m', 'Return 1y']:
    data_tmp.loc[data_tmp[i].isna() ,i] = data_tmp[i].min()
    
data_tmp.loc[data_tmp['Volatility'].isna(),'Volatility'] = data_tmp['Volatility'].median()

data_tmp.loc[data_tmp['EA ret'].isna(),'EA ret'] = data_tmp['EA ret'].median()
data_tmp.loc[data_tmp['EA Volatility'].isna(),'EA Volatility'] = data_tmp['Volatility'].median()/100

#Impute 100 day volatility for NANs
data_tmp.loc[data_tmp['1 Year Volatility'].isna(),'1 Year Volatility'] = data_tmp.loc[data_tmp['1 Year Volatility'].isna(),'Volatility']/100

#Impute median for NAN MAD
data_tmp.loc[data_tmp['1 Year MAD'].isna(),'1 Year MAD'] = data_tmp['1 Year MAD'].median()

In [34]:
## Impute EA std and calculate EA STD after everything has been imputed
data_tmp["EA ret std"] = data_tmp["EA ret"]/((data_tmp["EA Volatility"]*np.sqrt(3))/np.sqrt(252))

#Impute median for NAN EA STD
data_tmp.loc[data_tmp['EA ret std'].isna(),'EA ret std'] = data_tmp['EA ret std'].median()

In [35]:
### SAVE cleaned file into a CSV
save = "YES"
if save == "YES":
    data_date  = re.search(r'\d{4}-\d{2}-\d{2}', file_name)
    folder = '../clean_equity_data'
    exp_file_name = "clean_data_{}".format(data_date.group(0))+ ".csv"
    data_tmp.to_csv(os.path.join(folder, exp_file_name))